In [2]:
import simpy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import warnings
import random

In [123]:
RANDOM_SEED = 42
INTERVAL_CUSTOMERS = 10.0  # Generate new customers roughly every x seconds
CLIENTS_PER_TIME = 60/INTERVAL_CUSTOMERS
MIN_PATIENCE = 1  # Min. customer patience
MAX_PATIENCE = 3  # Max. customer patience
N_STEPS = 5000

TIME_IN_BANK_0 = 9
PROFIT_0 = 3 - 0.3
LOSS_0 = 1.1

TIME_IN_BANK_1 = 15
PROFIT_1 = 6 - 0.5
LOSS_1 = 2.5

PAYMENT_PER_WINDOW = 0.03
WINDOWS = 3

In [124]:
class BANK(object):
    def __init__(self, env):
        self.env = env
        self.window = simpy.Resource(env, capacity=WINDOWS)

    def service(self, client, client_type):
        if client_type:
            yield self.env.timeout(np.random.exponential(1/TIME_IN_BANK_1))
        else:
            yield self.env.timeout(np.random.exponential(1/TIME_IN_BANK_0))

In [125]:
def go_to_bank(env, client, client_type,  bank):
    arrival_time = env.now
    len_queue.append(len(bank.window.queue)) # длина очереди 
    clients_in_CMO.append(len(bank.window.queue) + len(bank.window.users)) # количество заявок в СМО
    clients_in_process.append(len(bank.window.users)) # число занятых каналов
    
    with bank.window.request() as request:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        # Wait for the counter or abort at the end of our tether
        results = yield request | env.timeout(patience)

        wait_time_in_queue.append(env.now - arrival_time)

        if request in results:
            yield env.process(bank.service(client, client_type))

        else:
            global clients_gone
            wait_times.append(env.now - arrival_time) 
            clients_gone += 1

In [128]:
def run_bank(env):
    bank = BANK(env)
    
    global clients_total
    while True:
        yield env.timeout(np.random.exponential(1/CLIENTS_PER_TIME))
        client_type = np.random.choice([0, 1], p=[0.75, 0.25])
        clients_total += 1
        clients_total_types[client_type] += 1
        env.process(go_to_bank(env, clients_total, client_type, bank))

In [129]:
clients_total_types = {0: 0, 1: 0}
clients_total, clients_gone = [0 for x in range(2)]
wait_times, len_queue, clients_in_CMO, clients_in_process, wait_time_in_queue = [[] for x in range(5)]

env = simpy.Environment()
env.process(run_bank(env))
env.run(until=N_STEPS)
print('''Имитационные результаты:
----------------------------------------------------------------
Вероятность того, что СМО свободна и может обслужить заявку = {}
Вероятность отказа = {}
Относительная пропускная способность = {}
Абсолютная пропускная способность = {}
Среднее число занятых каналов = {}
Средняя длина очереди = {}
Среднее время обслуживания в очереди = {}
Среднее число заявок в СМО = {}
Среднее время пребывания заявки в СМО = {}
'''.format(round(clients_in_process.count(0)/clients_total, 5),
           round(clients_gone/clients_total, 5),
           round(1 - clients_gone/clients_total, 5),
           round((clients_total-clients_gone)/emitation_time, 5),
           round(np.mean(clients_in_process), 5),
           round(np.mean(len_queue), 5),
           round(np.mean(wait_time_in_queue), 5),
           round(np.mean(clients_in_CMO), 5),
           round(np.mean(wait_times), 5)))

Имитационные результаты:
----------------------------------------------------------------
Вероятность того, что СМО свободна и может обслужить заявку = 0.54639
Вероятность отказа = 0.0
Относительная пропускная способность = 1.0
Абсолютная пропускная способность = 3.0015
Среднее число занятых каналов = 0.60586
Средняя длина очереди = 0.0059
Среднее время обслуживания в очереди = 0.00099
Среднее число заявок в СМО = 0.61176
Среднее время пребывания заявки в СМО = nan



In [96]:
# 
client_type = np.random.choice([0, 1], p=[0.75, 0.25])
clients_total[client_type] += 1

In [97]:
clients_total

{0: 20, 1: 6}

In [ ]:
def customer(env, name, counter, time_in_bank):
    """Customer arrives, is served and leaves."""
    arrive = env.now

In [60]:
interval = 10
print(random.expovariate(1.0 / interval))
np.random.exponential(interval)

8.32580071659295


4.426415068240226

In [ ]:
  
    with counter.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        results = yield req | env.timeout(patience)
        wait = env.now - arrive
        
        if req in results:
            # We got to the counter
            print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))

            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
        